In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv(r"C:\Users\ASUS\Desktop\DataSciencePurwadhika\Modul3\Hotel-Booking-Demand\data\raw\hotel_bookings.csv")
df.shape

(119390, 32)

In [78]:
display(df[['is_canceled', 'adr', 'reservation_status']].query("is_canceled == 0 & adr <= 0"))

len(df[['is_canceled', 'adr', 'reservation_status']].query("is_canceled == 0 & adr <= 0")) / len(df) * 100

,is_canceled,adr,reservation_status
0,0,0.0,Check-Out
1,0,0.0,Check-Out
125,0,0.0,Check-Out
167,0,0.0,Check-Out
168,0,0.0,Check-Out
...,...,...,...
118631,0,0.0,Check-Out
118762,0,0.0,Check-Out
118963,0,0.0,Check-Out
119102,0,0.0,Check-Out


1.4632716307898483

Insight:
- percentage of pesanan yang tidak dibatalkan tapi adr = 0, karena hanya 1.5% maka dilakukan drop untuk analisa selanjutnya

In [79]:
df.query("is_canceled == 1 & adr <= 0")

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
421,Resort Hotel,1,57,2015,July,29,15,0,2,2,...,No Deposit,305.0,NaN,0,Transient-Party,0.0,0,0,Canceled,2015-07-13
1539,Resort Hotel,1,304,2015,September,36,3,0,3,40,...,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
1545,Resort Hotel,1,297,2015,September,36,3,1,3,2,...,No Deposit,240.0,NaN,0,Transient,0.0,0,0,Canceled,2014-11-18
1587,Resort Hotel,1,333,2015,September,36,5,2,5,26,...,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
1643,Resort Hotel,1,336,2015,September,37,7,1,2,50,...,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104650,City Hotel,1,1,2017,February,5,1,0,0,1,...,No Deposit,NaN,153.0,0,Transient,0.0,0,1,No-Show,2017-02-01
105909,City Hotel,1,0,2017,April,16,18,0,1,1,...,No Deposit,NaN,45.0,0,Transient,0.0,0,2,Canceled,2017-04-18
106117,City Hotel,1,23,2017,February,8,21,0,0,2,...,No Deposit,14.0,NaN,0,Transient,0.0,0,0,Canceled,2017-02-21
106757,City Hotel,1,14,2017,May,21,24,0,1,1,...,No Deposit,NaN,45.0,0,Transient,0.0,0,1,Canceled,2017-05-22


In [80]:
# DataFrame tanpa is_canceled = 0 (tidak dibatalkan) and adr = 0 
data = df.query("~(is_canceled == 0 & adr <= 0)")

# DataFrame tanpa adr = 0 (memang benar di-cancel)
data = data.query("~(is_canceled == 1 & adr <= 0)")
data.shape

(117430, 32)

In [61]:
# cek duplicated data
print(f"""jumlah data duplikat {data.query("@data.duplicated()").shape[0]} baris""")

# drop duplicated data
data = data.drop_duplicates()
data.duplicated().sum()

jumlah data duplikat 31813 baris


0

<h1><center> <span style="color:#0ad0f7">Revenue Analysis per Room</span></center></h1>

In [81]:
rev_data = data['adr'].to_frame()

# 5% of adr per room (rule of thumb marketing cost = 5% x revenue (adr))
rev_data['cost_marketing_per_room'] = 0.11 * rev_data['adr'].abs()
rev_data

,adr,cost_marketing_per_room
2,75.00,8.2500
3,75.00,8.2500
4,98.00,10.7800
5,98.00,10.7800
6,107.00,11.7700
...,...,...
119385,96.14,10.5754
119386,225.43,24.7973
119387,157.71,17.3481
119388,104.40,11.4840


<h3><span style="color:#0ad0f7">Point Estimate</span></h3>

In [82]:
# rata-rata marketing cost
avg_marketing = rev_data['cost_marketing_per_room'].mean()
avg_marketing

11.388390000851569

Insight:
- rata-rata biaya marketing per room adalah **12 USD**

In [83]:
# rata-rata revenue tanpa marketing
avg_adr = rev_data['adr'].mean()
avg_adr

103.53081818955974

In [84]:
# rata-rata revenue dengan marketing
rev_data['adr'].mean() - avg_marketing

92.14242818870817

Insight:
- Average Daily Rate adalah **total biaya per kamar dibagi dengan jumlah hari menginap**.
- jika tidak mengadakan marketing pihak hotel akan mendapatkan rata-rata ADR sebesar **103 USD**, sedangkan jika mengadakan marketing pihak hotel akan mendapatkan rata-rata ADR sebesar **92 USD**.

<h3><span style="color:#0ad0f7">Confidence Interval For Mean</span></h3>

In [46]:
from scipy.special import boxcox, inv_boxcox
from scipy import stats

def normaltransform(df,cols):
    """Function to find CI for mean (if the data distribution is not normal)
    
    Args:
        df : DataFrame
        cols (string) : name of your column

    Return:
        lower, upper of CI for mean
    """
    # values of Wine in df1
    x = df[cols]

    # box-cox transform
    xt, lmbda = stats.boxcox(x)

    # 95% confidence interval of mean in a transformed scale
    lo_xt, hi_xt = stats.t.interval(1 - 0.05, len(xt) - 1, loc=np.mean(xt), scale=stats.sem(xt))
    lo_x, hi_x = inv_boxcox(lo_xt, lmbda), inv_boxcox(hi_xt, lmbda)

    # inverse transform
    lo_x, hi_x = inv_boxcox(lo_xt, lmbda), inv_boxcox(hi_xt, lmbda)

    return lo_x, hi_x

In [87]:
# rata-rata marketing cost
lowerMark, upperMark = normaltransform(rev_data, 'cost_marketing_per_room')
print(f"CI for mean marketing cost sebesar {lowerMark} USD sampai dengan {upperMark} USD")

CI for mean marketing cost sebesar 10.585488449805467 USD sampai dengan 10.640252729683498 USD


In [88]:
lowerMark1, upperMark1 = normaltransform(rev_data, 'adr')
print(f"CI for mean ADR (tanpa marketing) sebesar {lowerMark1} USD sampai dengan {upperMark1} USD")

CI for mean ADR (tanpa marketing) sebesar 96.23171339094063 USD sampai dengan 96.7295704811953 USD
